# Algorithme cryptographique RSA 

### Génération des clés RSA :
On a : Soit n = p * q où p et q sont des nombres premiers.
On calcule ensuite phi(n) = (p - 1)*(q - 1), ordre du groupe multiplicatif.
On a "e", un entier premier avec phi(n)
Puis d, un entier qui satisfait d*e = 1 (modulo phi(n))
==> d*e + u*phi(n) = 1 


### Pourquoi "e" est à 65537 ?
Pour que e soit valide dans RSA, il doit respecter ces conditions :
- e doit être premier avec φ(n) : gcd(e, φ(n)) = 1 (sinon on ne peut pas calculer d = e⁻¹ mod φ(n)).
- e doit être impair : Pour éviter d'avoir un d pair (qui affaiblirait RSA).
- e doit être assez petit : Pour accélérer le chiffrement (C = M^e mod n).

### Clé publique
n = p*q : module public
e : exposant public

### Clé privée
d = e^-1 (modulo phi(n))

### Nous devons respecter les étapes suivantes (but recherché avec des nombres de grande taille) :
Générer une clé RSA (n = p * q, e, d).

Factoriser n sur le GPU (trouver p, q en parallèle).

Calculer d (clé privée) en trouvant l'inverse modulaire de e.

### Documentation CUMP à voir sur le lien suivant :
https://github.com/skystar0227/CUMP



In [ ]:
!git clone https://github.com/skystar0227/CUMP.git

In [ ]:
!ls

In [ ]:
#include <iostream>
#include <cuda_runtime.h>
#include <cump.cimp.h>

#define THREADS_PER_BLOCK 256

In [ ]:
void generate_rsa_key(cump::bigint &n, cump::bigint &e, cump::bigint &d, cump::bigint p, cump::bigint q)
{
    n = p * q;
    cump::bigint phi = (p - 1) * (q - 1);
    e = cump::bigint(65537);  
    d = cump::mod_inv(e, phi);  
}

__global__ void factorize_kernel(cump::bigint n, cump::bigint *factors, int *found)
// n : entier à  factoriser de la clé publique RSA
// factors : tableau pour stocker p et q
// found : indicateur de recherche
{
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    cump::bigint p(tid + 2), q, rem;

    if (*found == 1) return;  
    // si un des autres threads utilisés a déjà  trouvé p et q, on arrête

    cump::mod(rem, n, p);   
    // stocke le resultat de n%p dans rem
    if (rem == cump::bigint(0)) { // si rem vaut 0 alors p est un diviseur de n
        q = n / p;
        factors[0] = p;
        factors[1] = q;
        *found = 1;
    }
}

void factorize_n_gpu(cump::bigint n, cump::bigint &p, cump::bigint &q) {
    cump::bigint *d_factors;
    int *d_found, h_found = 0;

    cudaMalloc((void **)&d_factors, 2 * sizeof(cump::bigint));
    cudaMalloc((void **)&d_found, sizeof(int));
    cudaMemcpy(d_found, &h_found, sizeof(int), cudaMemcpyHostToDevice);

    int num_blocks = (cump::size_in_bits(n) / THREADS_PER_BLOCK) + 1;
    factorize_kernel<<<num_blocks, THREADS_PER_BLOCK>>>(n, d_factors, d_found);
    cudaDeviceSynchronize();

    cudaMemcpy(&h_found, d_found, sizeof(int), cudaMemcpyDeviceToHost);
    if (h_found == 1) {
        cudaMemcpy(&p, &d_factors[0], sizeof(cump::bigint), cudaMemcpyDeviceToHost);
        cudaMemcpy(&q, &d_factors[1], sizeof(cump::bigint), cudaMemcpyDeviceToHost);
    }

    cudaFree(d_factors);
    cudaFree(d_found);
}

// VOIR POUR UNE AUTRE APPROCHE